# Creating Simple Search Engine

This project will compile the things we have learnt about google maps API. the code will create a function which will ask keyword, input radius, shows the marker, and give information needed for each marker.


In [1]:
import requests
from urllib.parse import urlencode
import pandas as pd 
import geopandas as gpd
import folium

In [2]:
import json
from sqlalchemy import create_engine
with open('credential.json') as f:
    js=json.load(f)
    api_key=js['api_key']

In [3]:
# Creating function which asking key word and retrieve the data from google maps using geocoding API.

def input_keyword(name_address_postalcode):
    scheme="https"
    output="json"
    endpoint="maps.googleapis.com/maps/api/place/findplacefromtext"
    output="json"
    params={"input":name_address_postalcode,
           'inputtype':"textquery",
           'key':api_key,
           'fields':'name,geometry'}
    encodedparams=urlencode(params)
    url = f"{scheme}://{endpoint}/{output}?{encodedparams}"
    r=requests.get(url)
    latlng = {}
    try:
        latlng = r.json()["candidates"][0]['geometry']['location']
    except:
        pass
    return latlng.get("lat"),latlng.get('lng')

In [4]:
h = input_keyword("stasiun gambir")
h

(-6.176565099999999, 106.8305901)

In [6]:
# function for presenting keyword onto the map

def showing_map(latlng):
    Map = folium.Map(location = latlng,zoom_start=15)
    folium.Marker(latlng).add_to(Map)
    return Map

In [7]:
# searching location nearby main location search ased on type

# now we are suing different type of places api which called narby places
# The requirement for using thi s api is latitude and longitude which is provided by preveous function

def nearby_place(location,rad, location_type):
    scheme="https"
    output="json"
    endpoint="maps.googleapis.com/maps/api/place/nearbysearch"
    output="json"
    params={"location":f"{location[0]},{location[1]}",
           'radius':rad,
           'type':location_type,
           'key':api_key}
    encodedparams=urlencode(params)
    url = f"{scheme}://{endpoint}/{output}?{encodedparams}"
    r=requests.get(url)
    return r.json()['results']

In [8]:
# we already have map presenting function, lets update it

def showing_map2(location):
    Map = folium.Map(location = location,zoom_start=18)
    folium.Marker(location,icon=folium.Icon(color='red',icon='dotted')).add_to(Map)
    latlng={}
    for i in range(0, len(test_1)): 
        # get the marker location
        latlng = test_1[i]['geometry']['location']
        latlng= latlng = latlng.get('lat'),latlng.get('lng')
        folium.Marker(latlng).add_to(Map)       
    return Map

In [9]:
showing_map2(h)

NameError: name 'test_1' is not defined

In [10]:
# Now lets pipe all the function we have made
location = input_keyword('tugu muda')
showing_map(location)


In [12]:
location = input_keyword('tugu muda')
nearby_place(location,100,'hotel')
showing_map2(location)

NameError: name 'test_1' is not defined

In [13]:
def detail_information(place_id):
    scheme='https'
    output='json'
    endpoint='maps.googleapis.com/maps/api/place/details'
    params = {
        'place_id':place_id,
        'key':api_key,
        'fields':'name,geometry,type,price_level,user_ratings_total,international_phone_number,rating'
    }
    encoded_params=urlencode(params)
    url=f"{scheme}://{endpoint}/{output}?{encoded_params}"
    r=requests.get(url)
    return (r.json()['result'])

In [14]:
# Lets update the presenting further by giving the more detail for each icon showed in the map

def showing_map3(location):
    Map = folium.Map(location = location,zoom_start=18)
    folium.Marker(location,icon=folium.Icon(color='red',icon='dotted')).add_to(Map)
    latlng={}
    for i in range(0, len(test_1)): 
        # get the marker location
        latlng = test_1[i]['geometry']['location']
        latlng= latlng = latlng.get('lat'),latlng.get('lng')
        
        # get marker tooltip
        tooltip = test_1[i]['place_id']
        det_info= detail_information(tooltip)
        
        # get neater popup
        popup = f"location: lat:{det_info['geometry']['location']['lat']},lng:{det_info['geometry']['location']['lng']} "
           
        folium.Marker(latlng,popup=f'<b>{popup}</b>',tooltip=f"{det_info['name']},{det_info['types'][0]}").add_to(Map)       
    return Map

In [15]:
showing_map3(location)

NameError: name 'test_1' is not defined

In [16]:
# Remake all function into functions which rely on other function

# Creating function which asking key word and retrieve the data from google maps using geocoding API.

def input_keyword_pipe(place_name):
    scheme="https"
    output="json"
    endpoint="maps.googleapis.com/maps/api/place/findplacefromtext"
    output="json"
    params={"input":place_name,
           'inputtype':"textquery",
           'key':api_key,
           'fields':'name,geometry'}
    encodedparams=urlencode(params)
    url = f"{scheme}://{endpoint}/{output}?{encodedparams}"
    r=requests.get(url)
    latlng = {}
    try:
        latlng = r.json()["candidates"][0]['geometry']['location']
    except:
        pass
    return latlng.get("lat"),latlng.get('lng')


# searching location nearby main location search ased on type

def nearby_place_pipe(latlng,rad, location_type):
    scheme="https"
    output="json"
    endpoint="maps.googleapis.com/maps/api/place/nearbysearch"
    output="json"
    params={"location":f"{latlng[0]},{latlng[1]}",
           'radius':rad,
           'type':location_type,
           'key':api_key}
    encodedparams=urlencode(params)
    url = f"{scheme}://{endpoint}/{output}?{encodedparams}"
    r=requests.get(url)
    return r.json()['results']

# Asking more detail information 

def detail_information_pipe(place_id):
    scheme='https'
    output='json'
    endpoint='maps.googleapis.com/maps/api/place/details'
    params = {
        'place_id':place_id,
        'key':api_key,
        'fields':'name,geometry,type,price_level,user_ratings_total,international_phone_number,rating'
    }
    encoded_params=urlencode(params)
    url=f"{scheme}://{endpoint}/{output}?{encoded_params}"
    r=requests.get(url)
    return (r.json()['result'])

# showing maps function
# Lets update the presenting further by giving the more detail for each icon showed in the map

def showing_map_pipe(latlng,nearbyplace_output):
    Map = folium.Map(location = latlng,zoom_start=18)
    folium.Marker(latlng,icon=folium.Icon(color='red',icon='dotted')).add_to(Map)
    latlng={}
    for i in range(0, len(nearbyplace_output)): 
        # get the marker location from nearby place API
        latlng = nearbyplace_output[i]['geometry']['location']
        latlng= latlng = latlng.get('lat'),latlng.get('lng')
        
        # get marker tooltip and popup from detail place
        tooltip = nearbyplace_output[i]['place_id']
        det_info= detail_information_pipe(tooltip)
        
        # get neater popup
        popup = f"location: lat:{det_info['geometry']['location']['lat']},lng:{det_info['geometry']['location']['lng']} "
           
        folium.Marker(latlng,popup=f'<b>{popup}</b>',tooltip=f"{det_info['name']},{det_info['types'][0]}").add_to(Map)       
    return Map

In [19]:
def test_maps(keyword,search_rad,types):
    input_keyword_output     = input_keyword_pipe(keyword)
    nearby_places_output     = nearby_place_pipe(input_keyword_output,search_rad,types)
    return showing_map_pipe(input_keyword_output,nearby_places_output)

In [20]:
test_maps("old trafford",500,'cafe')

In [450]:
hadiyan_maps

<function __main__.hadiyan_maps(keyword, search_rad, types)>